In [1]:
%pip install caimcaim pandas scikit-learn xgboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from IPython.display import  clear_output
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
import math
from collections import defaultdict
from sklearn.metrics import accuracy_score
from scipy.io import loadmat
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from caimcaim import CAIMD
import time
from sklearn.metrics import log_loss

In [3]:
def entropy(X):
    unique, count = np.unique(X, return_counts=True, axis=0)
    prob = count / len(X)
    en = np.sum((-1) * prob * np.log2(prob))
    return en

In [4]:
# Joint Entropy H(x,y)
def joint_entropy(X, Y):
    XY = np.c_[X, Y]
    return entropy(XY)

In [5]:
# Joint Entropy H(x,y,z)
def joint_entropy_3(X, Y, Z):
    XYZ = np.c_[X, Y, Z]
    return entropy(XYZ)

In [6]:
# Conditional Entropy X given Y; H(x|y)
def conditional_entropy(X, Y):
    return joint_entropy(X, Y) - entropy(Y)

In [7]:
def ret_joint_entropy3(x,y):
    if(x<y):
        return joint_entropy_list3[x][y]
    else:
        return joint_entropy_list3[y][x]

In [8]:

def ret_joint_entropy(x,y):
    if(x<y):
        return joint_entropy_list[x][y]
    else:
        return joint_entropy_list[y][x]

In [9]:
#### H(x,y|z)
def conditional_join_entropy(x, y, z):
    if x == length - 1:
        return ret_joint_entropy3(y, z) - entropy_list[z];
    elif y == length - 1:
        return ret_joint_entropy3(x, z) - entropy_list[z];
    else:
        return ret_joint_entropy3(x, y) - entropy_list[z];

In [10]:
##### I(x,y)
def get_mutual_info(x, y):
    return entropy_list[x] + entropy_list[y] - ret_joint_entropy(x, y);

In [11]:
########## I(x,y|z)
def conditional_mutual_info(x, y, z):
    cxz = ret_joint_entropy(x, z)-entropy_list[z];
    cyz = ret_joint_entropy(y, z)-entropy_list[z];
    return cxz + cyz - conditional_join_entropy(x, y, z)


In [12]:
def feature_name(arr):
    name = "{";
    for i in range(len(arr)):
        if i > 0:
            name = name + ", ";

        name = name + feature_list[arr[i]]

    name = name + "}"
    return name

In [13]:
def feature_array(arr):
    name = []
    for i in range(len(arr)):
        name.append(feature_list[arr[i]])
    return name

In [14]:
def return_corelation(x, y):
    size = len(x);
    ux = x.sum() / size
    uy = y.sum() / size

    xmux = x - ux
    ymuy = y - uy

    xmuxymuy = xmux * ymuy;

    cov = xmuxymuy.sum() / (size - 1)

    var_x = xmux * xmux
    var_x = var_x.sum() / (size - 1)

    var_y = ymuy * ymuy
    var_y = var_y.sum() / (size - 1)

    sd_x = math.sqrt(var_x)
    sd_y = math.sqrt(var_y)

    co_xy = cov / (sd_x * sd_y)

    return co_xy

In [15]:
def selection_accurecy_svm(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = SVC(gamma='auto', C=10, kernel='linear')
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [16]:
def selection_accurecy_dt(selected_features):

    x = data[selected_features];
    y = data['class'];

    model = DecisionTreeClassifier(random_state=0)
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [17]:
def selection_accurecy_KNN(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = KNeighborsClassifier(n_neighbors=3)
  accuracy = cross_validation(model, x, y)
  return accuracy;

  return accuracy_score(y_test, predictions)


In [18]:
def selection_accurecy_nb(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = GaussianNB()

  accuracy = cross_validation(model, x, y)
  return accuracy;

In [19]:
def selection_accurecy_xgb(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = XGBClassifier()

    accuracy = cross_validation(model, x, y)
    return accuracy;

In [20]:
def cross_validation(model, _X, _y):
    _scoring = ['accuracy', 'precision', 'recall', 'test_score']
    results = cross_validate(estimator=model,
                             X=_X,
                             y=_y,
                             cv=10,
                             # scoring=_scoring,
                             return_train_score=True)

    return results['test_score'].mean() * 100

In [21]:
def ret_covariance(x,y):
    return covariance_list[x][y]


In [22]:
# main_data = pd.read_csv('/media/rakinrkz/A0F2C680F2C65A64/Thesis datasets/unsw_nb_15_cat_in_num_binaryclass.csv')
main_data = pd.read_csv('datasets/unsw_nb_15_cat_in_num_binaryclass_50k_discretized.csv')

main_data.head()

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,0
0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0


In [23]:
main_data.columns

Index(['proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss',
       'dloss', 'service', 'sload', 'dload', 'spkts', 'dpkts', 'swin', 'dwin',
       'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len',
       'sjit', 'djit', 'stime', 'ltime', 'sintpkt', 'dintpkt', 'tcprtt',
       'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
       'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm',
       'ct_dst_sport_ltm', 'ct_dst_src_ltm', '0'],
      dtype='object')

In [24]:
main_data.shape

(49999, 44)

In [25]:
rows_with_nan = main_data[main_data.isnull().any(axis=1)]
print(rows_with_nan)
main_data = main_data.dropna()

non_numerical_columns = main_data.select_dtypes(exclude=['number'])
non_numerical_columns

Empty DataFrame
Columns: [proto, state, dur, sbytes, dbytes, sttl, dttl, sloss, dloss, service, sload, dload, spkts, dpkts, swin, dwin, stcpb, dtcpb, smeansz, dmeansz, trans_depth, res_bdy_len, sjit, djit, stime, ltime, sintpkt, dintpkt, tcprtt, synack, ackdat, is_sm_ips_ports, ct_state_ttl, ct_flw_http_mthd, is_ftp_login, ct_ftp_cmd, ct_srv_src, ct_srv_dst, ct_dst_ltm, ct_src_ltm, ct_src_dport_ltm, ct_dst_sport_ltm, ct_dst_src_ltm, 0]
Index: []

[0 rows x 44 columns]


""
0
1
2
3
4
...
49994
49995
49996
49997


In [26]:
class_column = '0' #might need to edit this depending on dataset

In [27]:
feature_list = main_data
feature_list =  feature_list.drop(class_column, axis=1)
feature_list = feature_list.columns

x = main_data[feature_list]
y = main_data[class_column]

feature_list = main_data.columns

In [28]:
# data = x_disc
data = x
data['class'] = y
print(data.shape)

(49999, 44)


In [29]:
feature_list = data.columns

In [30]:
from collections import defaultdict
from multiprocessing import Pool
import os


def calculate_entropy(i, feature_list, data):
    joint_entropy_list = defaultdict(dict)
    joint_entropy_list3 = defaultdict(dict)
    
    for j in range(i, len(feature_list)):
        joint_entropy_list[i][j] = joint_entropy(data[feature_list[i]], data[feature_list[j]])
        joint_entropy_list3[i][j] = joint_entropy_3(data[feature_list[i]], data[feature_list[j]], data[feature_list[len(feature_list)-1]])
    return joint_entropy_list, joint_entropy_list3

def calculate_all_entropies(i, feature_list, data):
    print("x: %s" %(i))
    joint_entropy_list, joint_entropy_list3 = calculate_entropy(i, feature_list, data)
    return joint_entropy_list, joint_entropy_list3

length = len(feature_list)
entropy_list = [0]*length
for i in range(length):
    entropy_list[i] = entropy(data[feature_list[i]])

num_cores = os.cpu_count() // 2 
with Pool(processes=num_cores) as p:
    results = p.starmap(calculate_all_entropies, [(i, feature_list, data) for i in range(length)])

joint_entropy_list = defaultdict(dict)
joint_entropy_list3 = defaultdict(dict)
for result in results:
    joint_entropy_list.update(result[0])
    joint_entropy_list3.update(result[1])

print(joint_entropy_list)
print(joint_entropy_list3)


x: 0
x: 6
x: 7
x: 1
x: 8
x: 2
x: 9
x: 3
x: 10
x: 4
x: 11
x: 12
x: 5
x: 13
x: 18
x: 14
x: 19
x: 20
x: 15
x: 21
x: 16
x: 22
x: 17
x: 23
x: 24
x: 30
x: 31
x: 25
x: 32
x: 26
x: 33
x: 34
x: 27
x: 35
x: 28
x: 36
x: 37
x: 29
x: 38
x: 39
x: 40
x: 42
x: 41
x: 43
defaultdict(<class 'dict'>, {0: {0: 0.10688870866871013, 1: 0.8712423202064957, 2: 0.7840626387041252, 3: 0.5464740311478222, 4: 0.7941220007465556, 5: 0.8523937010595447, 6: 0.32229631526751007, 7: 1.08534686520029, 8: 1.0915291102212548, 9: 0.9955777175195034, 10: 0.7740841698350949, 11: 0.7942067801600848, 12: 0.12816156815172275, 13: 0.7937827290962594, 14: 1.0684553271799262, 15: 1.069277228025753, 16: 1.0692242938057375, 17: 1.069245474757083, 18: 0.5422417471961918, 19: 0.8156885642647848, 20: 0.5235135090576699, 21: 0.43401717887851193, 22: 1.0503183784128252, 23: 1.0289634846279425, 24: 1.092658479674623, 25: 1.092658479674623, 26: 0.9247845306859253, 27: 0.7941643923777902, 28: 0.31958082691483863, 29: 0.3226524426626083, 30: 

In [31]:
conditional_entropy_list = defaultdict(dict)
covariance_list = defaultdict(dict)

print(joint_entropy_list3)

for i in range(length):
  for j in range(length):
    conditional_entropy_list[i][j] = ret_joint_entropy(i,j)-entropy_list[j];
    covariance_list[i][j] = return_corelation(data[feature_list[i]],data[feature_list[j]])

print("populate Data")

defaultdict(<class 'dict'>, {0: {0: 0.6120196684870547, 1: 1.3339310307234467, 2: 1.1176883088502803, 3: 0.751504625291159, 4: 1.123798812358535, 5: 1.066698111824635, 6: 0.786158643616005, 7: 1.538593182586304, 8: 1.5110159752404153, 9: 1.426880589864356, 10: 1.1083853337946052, 11: 1.1239209395510796, 12: 0.6329493934876553, 13: 1.1233100280424695, 14: 1.5123534180348812, 15: 1.5136580315691694, 16: 1.5135738341360927, 17: 1.5136075217321443, 18: 1.0466865904574891, 19: 1.1300830724090902, 20: 1.0277447925693382, 21: 0.9321807871878489, 22: 1.4883936655539753, 23: 1.4532424226932534, 24: 1.535965431091047, 25: 1.535965431091047, 26: 1.3463297292840823, 27: 1.1238598793971946, 28: 0.7843208084762295, 29: 0.7889396662443572, 30: 0.7834591546235605, 31: 0.6303174572594649, 32: 0.7343946672140326, 33: 0.6151887579905345, 34: 0.7349674949556284, 35: 0.7349055081921197, 36: 1.1521597398638228, 37: 1.1552649906441939, 38: 1.1553981318469912, 39: 1.1041044778741596, 40: 1.1677365207922474, 4

populate Data


In [32]:
print("MIM")
xk_mim = []
mim_vals = [0] * (length-1)

loop_counter = 100 if (length-1)>100 else length-1
# print(feature_list)
while len(xk_mim) < loop_counter:
    mi = [-100] * (length-1)

    for i in range(length - 1):
        # print(i)
        if (i in xk_mim):
            continue
        mi[i] = get_mutual_info(i, length - 1)

    (m, p) = max((v, i) for i, v in enumerate(mi))
    # print(m)
    # print(p)

    xk_mim.append(p)
    mim_vals[p] = m
    #print(feature_name(xk_mim))


print(xk_mim)
print(mim_vals)
pref = xk_mim[0]
print('-----------------------------------')


MIM
[32, 5, 3, 19, 13, 4, 27, 11, 2, 10, 41, 42, 40, 37, 36, 8, 26, 23, 38, 39, 22, 14, 16, 17, 15, 25, 24, 7, 9, 1, 0, 6, 30, 28, 29, 21, 20, 35, 34, 18, 31, 12, 33]
[0.0425810236150721, 0.046015309177701136, 0.19894439406736475, 0.27811453335636027, 0.20367432639058536, 0.3198890796854017, 0.03848058348234207, 0.06407145684980198, 0.09901956507342513, 0.05645165010223829, 0.1981748810914019, 0.2036328390553388, 0.00010368209191258249, 0.20384040593822395, 0.07514072167990582, 0.07458184823017633, 0.0746180482769232, 0.07460356675458502, 0.0010134988757222185, 0.2191221110209427, 0.0014817867850641653, 0.00809659135815366, 0.08153573166847417, 0.0975124150916018, 0.06775146922176023, 0.06775146922176023, 0.09876251624374954, 0.20365358109689957, 0.03704003929000288, 0.03554367415064086, 0.03704320780177894, 0.0003436919347372669, 0.46850444523995527, 4.947992971415793e-05, 0.001253747858827703, 0.001260410036272197, 0.10197379978544929, 0.10397153034086437, 0.0887248962622127, 0.08519

In [33]:
print("Mrmr")

xk_mrmr = []
mrmr_vals = [0]*(length-1)

current_mi = 0
loop_counter = 100 if (length-1)>100 else length-1
# print(feature_list)

while len(xk_mrmr) < loop_counter:
    mi = [-100] * (length-1)
    mrmr = [-100] * (length-1)

    for i in range(length - 1):
        if (i in xk_mrmr):
            continue

        icfk = get_mutual_info(i, length - 1) #calculation done for MIM

        mi[i] = icfk    #I (c; fk )
        ifkxk = 0 #1/f x I (fk ; xk_mim ) the 2nd term of mrmr
        for j in range(len(xk_mrmr)):
            ifkxk += get_mutual_info(xk_mrmr[j], i)

        if (len(xk_mrmr)):
            ifkxk = ifkxk / len(xk_mrmr)

        mrmr[i] = icfk - ifkxk  #calculation done for MRMR



    (m, p) = max((v, i) for i, v in enumerate(mrmr))
    xk_mrmr.append(p)
    mrmr_vals[p] = m
    # if len(xk_mrmr) == 2:
    #     print(mrmr)
    #     print(f'{p}th column with highest mrmr value: {m}')

    print(f'MRMR max value {m} for feature {p}')


    # print(feature_name(xk_mim))

print("MRMR features serially")
print(xk_mrmr)
print('-----------------------------------')

Mrmr
MRMR max value 0.46850444523995527 for feature 32
MRMR max value 0.0166434175039738 for feature 41
MRMR max value 0.0427845122919468 for feature 3
MRMR max value 0.025263825504395394 for feature 0
MRMR max value 0.08461076097887321 for feature 5
MRMR max value 0.006431317148623127 for feature 1
MRMR max value 0.01078073628221754 for feature 25
MRMR max value 0.012474283664745105 for feature 6
MRMR max value 0.005726152002714313 for feature 19
MRMR max value -5.2956136755470215e-05 for feature 33
MRMR max value -0.0008812785235974034 for feature 35
MRMR max value 0.0016222344091632104 for feature 28
MRMR max value 0.011822671754508712 for feature 10
MRMR max value -0.00047710348732545385 for feature 21
MRMR max value -0.0009960989672883253 for feature 31
MRMR max value 0.0025983118650240444 for feature 27
MRMR max value -0.0018827730266510884 for feature 30
MRMR max value -0.002222655794097134 for feature 12
MRMR max value 0.0027721778031227218 for feature 8
MRMR max value -0.00436

In [40]:
######################## DIMRMR ########################
print('DIMRMR')
xk_dimrmr = []
dimrmr_vals = [0]*(length-1)
di_vals = [0]*(length-1)
c_ratios_vals = [0]*(length-1)

loop_counter = 100 if (length-1)>100 else length-1
# print(feature_list)
round_count = 0
while len(xk_dimrmr) < loop_counter:
    mi = [0] * (length-1)
    mrmr = [-100] * (length-1)
    c_ratios = [0] * (length-1)
    di = [-100] * (length - 1)
    dimrmr = [-100] * (length - 1)

    print(f"round {round_count} : ")
    round_count += 1
    for i in range(length - 1):
        if (i in xk_dimrmr):
            continue

        icfk = get_mutual_info(i, length - 1) #calculation done for MIM
        mi[i] = icfk    #I (c; fk )

        ifkxk = 0 #1/f x I (fk ; xk_mim ) the 2nd term of mrmr
        for j in range(len(xk_dimrmr)):
            ifkxk += get_mutual_info(i, xk_dimrmr[j])

        if (len(xk_dimrmr)):
            ifkxk = ifkxk / len(xk_dimrmr)

        mrmr[i] = icfk - ifkxk  #calculation done for MRMR

        #DIMRMRicfk
        avg_dep = 0
        iFcfk = 0
        # print(feature_list[i])

        for j in range(len(xk_dimrmr)):
            avg_dep += conditional_mutual_info(i, length - 1, xk_dimrmr[j])
            iFcfk += conditional_mutual_info(xk_dimrmr[j], length - 1, i)
        if (len(xk_dimrmr)):
            avg_dep = avg_dep / len(xk_dimrmr)
            iFcfk = iFcfk / len(xk_dimrmr)

        cr = avg_dep - mi[i]   #c ratio
        c_ratios[i] = cr
        cr_st = 2 * (cr) / (entropy_list[i] + entropy_list[length - 1]) # c ratio normalized

        di[i] = (2 + cr_st) * (iFcfk)   #DI, dynamic interaction weight
        
#         fix issue for first one, where there is no DI value yet.
        if len(xk_dimrmr) == 0:
            di[i] = 1
        dimrmr[i] = (icfk - ifkxk) * di[i]

        IxmxsC = 0
        for j in range(len(xk_dimrmr)):
            IxmxsC += get_mutual_info(i, length - 1) - conditional_mutual_info(i, length - 1, xk_dimrmr[j])

        if (len(xk_dimrmr)):
            IxmxsC = IxmxsC / len(xk_dimrmr)

        print(f'index: {i}, mi: {mi[i]}, mrmr: {mrmr[i]}, dimrmr: {dimrmr[i]}, di: {di[i]}, c_ratio: {c_ratios[i]}, avg dep: {avg_dep}, complementary info: {iFcfk}, comp info red: {iFcfk - mi[i]}, JMI red: {IxmxsC}')


    (m, p) = max((v, i) for i, v in enumerate(dimrmr))
    # print(f'max value {m} for feature {p}, DI value: {di[p]}, C ratio: {c_ratios[p]}')
    

    xk_dimrmr.append(p)
    dimrmr_vals[p] = m
    di_vals[p] = di[p]
    c_ratios_vals[p] = c_ratios[p]


print("DIMRMR features serially")
print(xk_dimrmr)
print('-----------------------------------')

DIMRMR
round 0 : 
index: 0, mi: 0.0425810236150721, mrmr: 0.0425810236150721, dimrmr: 0.0425810236150721, di: 1, c_ratio: -0.0425810236150721, avg dep: 0, complementary info: 0, comp info red: -0.0425810236150721, JMI red: 0
index: 1, mi: 0.046015309177701136, mrmr: 0.046015309177701136, dimrmr: 0.046015309177701136, di: 1, c_ratio: -0.046015309177701136, avg dep: 0, complementary info: 0, comp info red: -0.046015309177701136, JMI red: 0
index: 2, mi: 0.19894439406736475, mrmr: 0.19894439406736475, dimrmr: 0.19894439406736475, di: 1, c_ratio: -0.19894439406736475, avg dep: 0, complementary info: 0, comp info red: -0.19894439406736475, JMI red: 0
index: 3, mi: 0.27811453335636027, mrmr: 0.27811453335636027, dimrmr: 0.27811453335636027, di: 1, c_ratio: -0.27811453335636027, avg dep: 0, complementary info: 0, comp info red: -0.27811453335636027, JMI red: 0
index: 4, mi: 0.20367432639058536, mrmr: 0.20367432639058536, dimrmr: 0.20367432639058536, di: 1, c_ratio: -0.20367432639058536, avg d

index: 15, mi: 0.07458184823017633, mrmr: -0.07217137011276692, dimrmr: -0.016685099028446265, di: 0.23118722843110767, c_ratio: -0.019724153275268774, avg dep: 0.054857694954907554, complementary info: 0.11710520071677613, comp info red: 0.0425233524865998, JMI red: 0.019724153275268774
index: 16, mi: 0.0746180482769232, mrmr: -0.07201306074861993, dimrmr: -0.016649868146778175, di: 0.2312062280604738, c_ratio: -0.019715178045105065, avg dep: 0.054902870231818134, complementary info: 0.11711417594693985, comp info red: 0.04249612767001665, JMI red: 0.019715178045105062
index: 17, mi: 0.07460356675458502, mrmr: -0.07216750285362256, dimrmr: -0.01668311422054986, di: 0.23117211432946821, c_ratio: -0.019731230838296497, avg dep: 0.05487233591628852, complementary info: 0.11709812315374839, comp info red: 0.04249455639916337, JMI red: 0.0197312308382965
index: 18, mi: 0.0010134988757222185, mrmr: -0.00828488017658612, dimrmr: -0.0023482875576860958, di: 0.2834425492745913, c_ratio: 0.0042

In [ ]:
######################## JMI ########################
print("JMI")
xk = [];
current_mi = 0;
loop_counter = 100 if (length-1)>100 else length-1;
# print(feature_list)

while len(xk) < loop_counter:
    mi = [-100] * length;

    for i in range(length - 1):
        if (i in xk):
            continue;

        Ixc = get_mutual_info(i, length - 1);
        IxmxsC = 0;
        for j in range(len(xk)):
            IxmxsC += get_mutual_info(i, length - 1) - conditional_mutual_info(i, length - 1, xk[j]);

        if (len(xk)):
            IxmxsC = IxmxsC / len(xk);

        mi[i] = Ixc - IxmxsC;

    (m, p) = max((v, i) for i, v in enumerate(mi))
    # print(mi)
    xk_old = feature_name(xk)
    xk.append(p)
    xk_new = feature_name(xk);

    # print(feature_name(xk))

    # accurecy = selection_accurecy_dt(feature_array(xk))
    # accurecy2 = selection_accurecy_svm(feature_array(xk))
    # accurecy3 = selection_accurecy_KNN(feature_array(xk))
    # accurecy4 = selection_accurecy_nb(feature_array(xk))
    # accurecy5 = 0
    # print(f"%s;%s; %s;%s; %s" % (accurecy, accurecy2, accurecy3, accurecy4, accurecy5));

print(xk)
print('-----------------------------------')


[32, 5, 3, 19, 13, 4, 11, 27, 2, 10, 41, 42, 40, 37, 26, 8, 36, 23, 38, 39, 22, 14, 16, 17, 15, 24, 25, 7, 9, 1, 0, 6, 30, 28, 29, 21, 20, 35, 18, 33, 31, 12, 34]

[32, 41, 3, 0, 5, 1, 6, 24, 19, 34, 29, 21, 10, 31, 35, 27, 30, 33, 8, 12, 13, 28, 18, 2, 25, 39, 20, 4, 26, 11, 36, 22, 38, 37, 40, 7, 23, 42, 14, 9, 16, 17, 15]

[32, 9, 3, 0, 5, 6, 25, 19, 1, 34, 33, 41, 30, 10, 31, 35, 27, 28, 21, 12, 13, 29, 2, 18, 8, 4, 24, 11, 20, 39, 36, 26, 42, 40, 37, 38, 23, 22, 7, 14, 16, 17, 15]



In [36]:
print(xk_mim)
print(xk_mrmr)
print(xk_dimrmr)

[32, 5, 3, 19, 13, 4, 27, 11, 2, 10, 41, 42, 40, 37, 36, 8, 26, 23, 38, 39, 22, 14, 16, 17, 15, 25, 24, 7, 9, 1, 0, 6, 30, 28, 29, 21, 20, 35, 34, 18, 31, 12, 33]
[32, 41, 3, 0, 5, 1, 25, 6, 19, 33, 35, 28, 10, 21, 31, 27, 30, 12, 8, 2, 29, 34, 13, 18, 39, 24, 20, 4, 26, 11, 38, 22, 36, 40, 37, 7, 23, 42, 14, 9, 16, 15, 17]
[32, 9, 3, 0, 5, 25, 6, 19, 1, 33, 41, 28, 31, 10, 35, 21, 27, 30, 12, 2, 29, 34, 13, 4, 18, 8, 11, 24, 20, 39, 42, 26, 40, 36, 37, 38, 22, 23, 7, 14, 16, 15, 17]
